In [1]:
import sys
import re
sys.path.append("/uscms/home/sbrightt/nobackup/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/python_analysis/analysisTools")
import plotTools as ptools
import matplotlib.pyplot as plt
import numpy as np
from coffea import hist
from coffea import util
import importlib
import os

ptools.setDefaultStyle()

In [2]:
histoLoc = "/uscms_data/d3/sbrightt/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/python_analysis/outputs/signal_2018_aEM"
hnames = ["ele_match_matrix","ele_nMatch_matrix","ele_match_class",
          "matched_ele_kinematics","matched_ele_gen_kinematics","matched_ele_trackHits","matched_ele_trackQual","matched_ele_gen_displacement"]
histos = ptools.loadHistosFromFiles(histoLoc,hnames)
info, masses, cts = ptools.getSampleInfo(histos,hname="ele_match_matrix")

In [3]:
# plotting efficiency "match matrices"
match_mtx = histos["ele_match_matrix"]
nMatch_mtx = histos["ele_nMatch_matrix"]
matchClass = histos["ele_match_class"]

sets = ['RL','RLC']
schemes = ['unique','nearUnique']
types_bySet = {'RL':['Reg-Reg','Low-Reg','Low-Low','all'],'RLC':['Reg-Reg','Low-Reg','Low-Low','Reg-Cand','Low-Cand','all']}
patch_opts = {"cmap":"cool"}
text_opts = {"color":"black","fontweight":"bold","fontsize":14}

for pt in masses:
    for ct in cts:
        m, dm = pt.split("-")[0], pt.split("-")[1]
        samp = info[pt][ct]
        outdir = 'plots/{0}-{1}/gen_match_info/ct-{2}/'.format(m,dm,ct)
        os.makedirs(outdir,exist_ok=True)
        title = r"$m_\chi = {0}$, $\Delta m_\chi = {1}$, $c\tau = {2}$".format(m,dm,ct)
        
        # 1 - plotting "match class" matrix (i.e. no match, unique match, matched to same)
        h_matchClass_mtx = matchClass.integrate("sample",int_range=samp)
        plt.figure()
        hist.plot1d(h_matchClass_mtx,overlay="set",density=True)
        plt.ylim([0,1])
        plt.xticks(np.arange(0.5,4.5,step=1),labels=["No Matches","One Match","Degenerate","Unique"])
        plt.title(title)
        plt.savefig(outdir+"matchClass.pdf")
        plt.close()
        
        for st in sets:
            outdir = 'plots/{0}-{1}/gen_match_info/ct-{2}/{3}/'.format(m,dm,ct,st)
            os.makedirs(outdir,exist_ok=True)
            title = r"$m_\chi = {0}$, $\Delta m_\chi = {1}$, $c\tau = {2}$, Set: {3}".format(m,dm,ct,st)
            
            # 2 - plotting matrix of number of reco objects matched to ele/pos per event
            h_nMatch_matrix = nMatch_mtx.integrate("sample",int_range=samp).integrate("set",int_range=st)
            plt.figure()
            hist.plot2d(h_nMatch_matrix,xaxis="nEmatch",patch_opts=patch_opts,text_opts=text_opts,ax=plt.gca(),density=True)
            plt.title(title)
            plt.savefig(outdir+"nMatch_matrix_{0}.pdf".format(st))
            plt.close()
            
            for scm in schemes:
                title = r"$m_\chi = {0}$, $\Delta m_\chi = {1}$, $c\tau = {2}$, Scheme: {3}, Set: {4}".format(m,dm,ct,scm,st)
                
                h_match_mtx = match_mtx.integrate("sample",int_range=samp).integrate("set",int_range=st).integrate("scheme",scm)
                plt.figure()
                hist.plot2d(h_match_mtx,xaxis="Etype",patch_opts=patch_opts,text_opts=text_opts,ax=plt.gca(),density=True)
                plt.xticks(np.arange(0.5,4.5,step=1),labels=["No Match","Regular",r"Low $p_T$","Candidate"])
                plt.yticks(np.arange(0.5,4.5,step=1),labels=["No Match","Regular",r"Low $p_T$","Candidate"],rotation=90)
                plt.title(title)
                plt.savefig(outdir+"match_mtx_{0}_{1}.pdf".format(scm,st))
                plt.close()

In [5]:
matched_ele_kin = histos["matched_ele_kinematics"]
matched_ele_gen_kin = histos["matched_ele_gen_kinematics"]
matched_ele_trackHits = histos["matched_ele_trackHits"]
matched_ele_trackQual = histos["matched_ele_trackQual"]
matched_ele_gen_disp = histos["matched_ele_gen_displacement"]

sets = ['RL','RLC']

#doing ct overlay
for pt in masses:
    for st in sets:
        m, dm = pt.split("-")[0], pt.split("-")[1]
        samp = info[pt][ct]
        samps_ct = [info[pt][ct] for ct in cts]
        outdir = 'plots/ct-overlays/gen_match_info/{0}-{1}/'.format(m,dm,ct)
        os.makedirs(outdir,exist_ok=True)
        title = r"Matched $e^+/e^-$, $m_\chi = {0}$, $\Delta m_\chi = {1}$".format(m,dm)
        
        h_pt = matched_ele_kin.integrate("set",int_range=st).sum("eta","phi",overflow="all")
        h_eta = matched_ele_kin.integrate("set",int_range=st).sum("pt","phi",overflow="all")
        h_phi = matched_ele_kin.integrate("set",int_range=st).sum("eta","pt",overflow="all")
        h_gen_pt = matched_ele_gen_kin.integrate("set",int_range=st).sum("eta","phi",overflow="all")
        h_gen_eta = matched_ele_gen_kin.integrate("set",int_range=st).sum("pt","phi",overflow="all")
        h_gen_phi = matched_ele_gen_kin.integrate("set",int_range=st).sum("eta","pt",overflow="all")
        h_nTkHits = matched_ele_trackHits.integrate("set",int_range=st).sum("numPixHits","numStripHits",overflow="all")
        h_nStpHits = matched_ele_trackHits.integrate("set",int_range=st).sum("numPixHits","numTrkHits",overflow="all")
        h_nPixHits = matched_ele_trackHits.integrate("set",int_range=st).sum("numTrkHits","numStripHits",overflow="all")
        h_chi2 = matched_ele_trackQual.integrate("set",int_range=st).sum("trkIso",overflow="all")
        h_trkIso = matched_ele_trackQual.integrate("set",int_range=st).sum("chi2",overflow="all").rebin("trkIso",2)
        h_vxy = matched_ele_gen_disp.integrate("set",int_range=st).sum("vz",overflow="all").rebin("vxy",16)
        h_vz = matched_ele_gen_disp.integrate("set",int_range=st).sum("vxy",overflow="all").rebin("vz",16)
        
        # plotting pT
        plt.figure()
        plt.yscale("log")
        hist.plot1d(h_pt,overlay="sample",order=samps_ct,density=True)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.savefig(outdir+"matched_ele_pt.pdf")
        plt.close()
        
        # plotting eta
        plt.figure()
        hist.plot1d(h_eta,overlay="sample",order=samps_ct,density=True)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.savefig(outdir+"matched_ele_eta.pdf")
        plt.close()
        
        # plotting phi
        plt.figure()
        hist.plot1d(h_phi,overlay="sample",order=samps_ct,density=True)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.savefig(outdir+"matched_ele_phi.pdf")
        plt.close()
        
        # plotting gen pT
        plt.figure()
        plt.yscale("log")
        hist.plot1d(h_gen_pt,overlay="sample",order=samps_ct,density=True)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.xlabel("Generated $p_T$ [GeV]")
        plt.savefig(outdir+"matched_ele_gen_pt.pdf")
        plt.close()
        
        # plotting gen eta
        plt.figure()
        hist.plot1d(h_gen_eta,overlay="sample",order=samps_ct,density=True)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.xlabel("Generated $\eta$")
        plt.savefig(outdir+"matched_ele_gen_eta.pdf")
        plt.close()
        
        # plotting gen phi
        plt.figure()
        hist.plot1d(h_gen_phi,overlay="sample",order=samps_ct,density=True)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.xlabel("Generated $\phi$")
        plt.savefig(outdir+"matched_ele_gen_phi.pdf")
        plt.close()
        
        # plotting track hits
        plt.figure()
        hist.plot1d(h_nTkHits,overlay="sample",order=samps_ct,density=True)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.savefig(outdir+"matched_ele_nTkHits.pdf")
        plt.close()
        
        # plotting strip hits
        plt.figure()
        hist.plot1d(h_nStpHits,overlay="sample",order=samps_ct,density=True)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.savefig(outdir+"matched_ele_nStpHits.pdf")
        plt.close()
        
        # plotting pixel hits
        plt.figure()
        hist.plot1d(h_nPixHits,overlay="sample",order=samps_ct,density=True)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.savefig(outdir+"matched_ele_nPixHits.pdf")
        plt.close()
        
        # plotting chi2
        plt.figure()
        plt.yscale("log")
        hist.plot1d(h_chi2,overlay="sample",order=samps_ct,density=True)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.savefig(outdir+"matched_ele_chi2.pdf")
        plt.close()
        
        # plotting trkIso
        plt.figure()
        plt.yscale("log")
        hist.plot1d(h_trkIso,overlay="sample",order=samps_ct,density=True)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.savefig(outdir+"matched_ele_trkIso.pdf")
        plt.close()
        
        # plotting vxy
        plt.figure()
        plt.yscale("log")
        hist.plot1d(h_vxy,overlay="sample",order=samps_ct,density=True)
        plt.xlim([0,25])
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.savefig(outdir+"matched_ele_vxy.pdf")
        plt.close()
        
        # plotting vz
        plt.figure()
        plt.yscale("log")
        hist.plot1d(h_vz,overlay="sample",order=samps_ct,density=True)
        plt.xlim([0,25])
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.title(title)
        plt.savefig(outdir+"matched_ele_vz.pdf")
        plt.close()

/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/c